In [19]:
'''
Functions from in-class exercises
'''
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]
    
def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

In [8]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [9]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


### IMPLEMENTING MINI-BATCH GRADIENT DESCENT (WITHOUT DP FOR NOW)
#### Steps to follow to implement the mini-batch gradient descent (without dp):
1. Define a function that splits data into mini-batches (subsets of the whole dataset)
2. Define loss function that measures how good our model is.
3. Define gradient function. The gradient is a vector that indicates the rate of change of the loss in each direction.
4. Define avg_grad function that computes the average gradient over each mini-batch.
5. Define gradient_descent function that computes gradient using mini-batches for each iteration

In [10]:
def split_to_mini_batches(X, y, batch_size):
    # shuffling the data before creating mini_batches to prevent the model 
    # from learning possible patterns + each batch might contain more "diversified"
    # information. 
    
    shuffled_data = np.random.permutation(X.shape[0])
    randomized_X = X[shuffled_data]
    randomized_Y = y[shuffled_data]

    mini_batches = []
    for i in range(0,X.shape[0],batch_size):
        mini_batches.append((randomized_X[i:i+batch_size], randomized_Y[i:i+batch_size]))
        
    return mini_batches

In [11]:
'''
Functions taken from in-class-exercise 10.28.24
'''

# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

#Vectorized version of gradient calculation by wanglun1996. 
#Github: https://github.com/sunblaze-ucb/dpml-benchmark/blob/master/lossfunctions/logistic_regression.py#L12

def gradient_vectorized(theta, x, y, lambda_param=None):
    """
    Gradient function for logistic regression without regularization.
    Based on the above logistic_regression
    """
    exponent = y * (x.dot(theta))
    gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (x.shape[0])

    # Reshape to handle case where x is csr_matrix
    gradient_loss.reshape(theta.shape)

    return gradient_loss

def avg_grad(theta, X, y):

    #All_grads is a list of vectors, with each vector of length 104
    all_grads = [gradient(theta,X[i],y[i]) for i in range(len(X))] #one gradient per example in the data

    #Compute the column-wise average
    avg_grad = np.mean(all_grads,axis=0)
    
    return avg_grad

In [12]:
'''
Original function gradient_descent() taken from in-class-exercise 10.28.24 and modifying it
iterate over the mini-batches instead of over the entire dataset (full-batch)
'''
def mini_batch_gradient_descent(epochs, batch_size):
    #Step 1: initalize all thetas
    theta = [0 for _ in range(X_train.shape[1])] #Initial model

    #Step 2: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

    #Step 3: iterate for each num samples in training set (training_set = mini batch)
        for X_train_batch, y_train_batch in mini_batches:
            theta = theta - avg_grad(theta, X_train_batch, y_train_batch)
            
    return theta

theta = mini_batch_gradient_descent(50 , 64)
theta

array([ 1.23279202e-01, -7.14700205e-01, -3.98762690e-01, -4.81565631e-01,
       -1.13904947e+00, -8.40299661e-01, -1.52616247e+00, -1.07674666e+00,
       -9.24988667e-01, -7.34156256e-01, -1.41655830e+00, -8.68369648e-01,
       -1.20690537e+00, -1.23996151e+00,  5.86888630e-02,  4.75339865e-02,
        6.06300932e-01,  1.22419435e+00, -1.25939668e-01,  8.45288076e-01,
       -1.47567226e+00,  1.45467005e+00, -1.44638838e-01, -1.51631984e+00,
        1.39001062e+00,  9.68337294e-01, -1.19342615e+00, -1.77212941e+00,
       -1.50414323e+00, -1.34959021e+00, -2.90710295e-01, -6.91265868e-02,
       -1.73704536e-01,  5.11206181e-01, -1.17152379e+00, -8.97557892e-01,
       -4.70217206e-01, -1.17107977e+00, -1.75641089e+00,  4.40881086e-01,
        2.46980468e-01, -5.80628062e-02,  2.91677895e-01, -4.09612784e-01,
       -9.94349181e-01, -4.39699125e-01, -1.36795037e+00, -1.57288268e+00,
       -7.26856864e-01,  1.24477290e-01, -1.24508433e+00, -6.88638801e-01,
       -1.06288080e+00, -

In [13]:
'''
Functions taken from in-class-exercise 10.28.24
'''
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias) #this is the dot product and take the sign. 
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2) #computing L2 norm 
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

#theta = [-.1 for _ in range(104)]
accuracy(theta)

0.8258513931888545

### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH (EPSILON)- DP

In [14]:
def epsilon_noisy_gradient_descent(epochs, epsilon, batch_size):
    #Step 1: initalize all thetas 
    theta = [0 for _ in range(X_train.shape[1])]

    #Step 2: splitting the epsilon and choosing sensitivity
    epsilon_i = epsilon/epochs
    sensitivity = 1 #?
    
    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [gradient(theta,X_train_batch[i],y_train_batch[i]) for i in range(len(X_train_batch))]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(all_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = laplace_mech_vec(grad_sum,sensitivity=sensitivity,epsilon=epsilon_i)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) 
            
            theta = theta - noisy_grad
    
    return theta

theta = epsilon_noisy_gradient_descent(10, 1.0, 64) #a smaller epsilon, accuracy is not as good. Noise can make the model worse. 
                                                # If we increase iterations, it will make up for it. 
print('Final accuracy:', accuracy(theta))

Final accuracy: 0.739938080495356


### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH (EPSILON,DELTA)- DP

In [15]:
def epsilon_delta_noisy_gradient_descent(epochs, epsilon, delta, batch_size):
    #Step 1: initalize all thetas 
    theta = [0 for _ in range(X_train.shape[1])]

    #Step 2: splitting the epsilons and delta over the num of iterations/epochs.
    epsilon_i = epsilon/epochs
    delta_i = delta/epochs

    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [gradient(theta,X_train_batch[i],y_train_batch[i]) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_vec(grad_sum,sensitivity=b,epsilon=epsilon_i,delta=delta_i)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) 
            
            theta = theta - noisy_grad
    
    return theta

theta = epsilon_delta_noisy_gradient_descent(10, 1.0, 1e-5,64) #a smaller epsilon, accuracy is not as good. Noise can make the model worse. 
                                                # If we increase iterations, it will make up for it. 
print('Final accuracy:', accuracy(theta))

/var/folders/yt/mgj8y39x4xvdgz8vbdv_wkyw0000gn/T/ipykernel_53482/1055718158.py:15: RuntimeWarning: overflow encountered in exp
  return - (yi*xi) / (1+np.exp(exponent))


Final accuracy: 0.7214727996461743


In [18]:
def vectorized_epsilon_delta_noisy_gradient_descent(epochs, epsilon, delta, batch_size):
    #Step 1: initalize all thetas 
    theta = [0 for _ in range(X_train.shape[1])]

    #Step 2: splitting the epsilons and delta over the num of iterations/epochs.
    epsilon_i = epsilon/epochs
    delta_i = delta/epochs

    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [gradient_vectorized(theta,X_train_batch[i],y_train_batch[i]) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_vec(grad_sum,sensitivity=b,epsilon=epsilon_i,delta=delta_i)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) 
            
            theta = theta - noisy_grad
    
    return theta

theta = vectorized_epsilon_delta_noisy_gradient_descent(10, 1.0, 1e-5,64) #a smaller epsilon, accuracy is not as good. Noise can make the model worse. 
                                                # If we increase iterations, it will make up for it. 
print('Final accuracy:', accuracy(theta))

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH RÉNYI DP

In [31]:
'''
Original functions taken from homework assignment 9
'''
def mini_batch_noisy_gradient_descent_RDP(epochs, epsilon_bar, alpha, batch_size):
    #Step 1: initalize all thetas 
    theta = [0 for _ in range(X_train.shape[1])]


    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [gradient(theta,X_train_batch[i],y_train_batch[i]) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_RDP_vec(grad_sum,sensitivity=b,alpha=alpha,epsilon=epsilon_bar)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) #MAYBE DO LEN(MINI_BATCH)
            
            theta = theta - noisy_grad
    
    return theta

theta = mini_batch_noisy_gradient_descent_RDP(10, 0.1, 20,64) 
                                                
print('Final accuracy:', accuracy(theta))

Final accuracy: 0.7733303847854931


In [ ]:
'''
Original functions taken from homework assignment 9
'''
def vectorized_mini_batch_noisy_gradient_descent_RDP(epochs, epsilon_bar, alpha, batch_size):
    #Step 1: initalize all thetas 
    theta = [0 for _ in range(X_train.shape[1])]


    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [gradient_vectorized(theta,X_train_batch[i],y_train_batch[i]) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_RDP_vec(grad_sum,sensitivity=b,alpha=alpha,epsilon=epsilon_bar)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) #MAYBE DO LEN(MINI_BATCH)
            
            theta = theta - noisy_grad
    
    return theta

theta = vectorized_mini_batch_noisy_gradient_descent_RDP(10, 0.1, 20,64) 
                                                
print('Final accuracy:', accuracy(theta))

### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH zCDP

In [38]:
'''
Original functions taken from homework assignment 9
'''
def mini_batch_noisy_gradient_descent_zCDP(epochs, rho, batch_size):
    #IDEA: copy noisy_gradient_descent but use gaussian_mech_zCDP_vec to compute the noisy_gradient_sum
    
    #from the noisy_gradient_descent function provided above: 
    theta = np.zeros(X_train.shape[1])
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [gradient(theta,X_train_batch[i],y_train_batch[i]) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_zCDP_vec(grad_sum,sensitivity=b,rho=rho)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) #MAYBE DO LEN(MINI_BATCH)
            
            theta = theta - noisy_grad
    
    return theta

theta = mini_batch_noisy_gradient_descent_zCDP(10, 0.1, 64)
print('Final accuracy:', accuracy(theta))

Final accuracy: 0.7739938080495357


In [20]:
'''
Original functions taken from homework assignment 9
'''
def vectorized_mini_batch_noisy_gradient_descent_zCDP(epochs, rho, batch_size):
    #IDEA: copy noisy_gradient_descent but use gaussian_mech_zCDP_vec to compute the noisy_gradient_sum
    
    #from the noisy_gradient_descent function provided above: 
    theta = np.zeros(X_train.shape[1])
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [gradient_vectorized(theta,X_train_batch[i],y_train_batch[i]) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_zCDP_vec(grad_sum,sensitivity=b,rho=rho)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) #MAYBE DO LEN(MINI_BATCH)
            
            theta = theta - noisy_grad
    
    return theta

theta = vectorized_mini_batch_noisy_gradient_descent_zCDP(10, 0.1, 64)
print('Final accuracy:', accuracy(theta))

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)